In [1]:
import pandas as pd 
import numpy as np
import public_timeseries_testing_util as optiver2023
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, unpack_sequence, unpad_sequence
import torch
from tqdm import trange,tqdm
import torch.nn as nn 
import torch.optim as optim
import wandb
import torch_classes

In [2]:
env = optiver2023.make_env()
iter_test = env.iter_test()

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [4]:
train = pd.read_csv('train.csv')
train.head()
train.date_id.value_counts()

date_id
480    11000
353    11000
363    11000
362    11000
360    11000
       ...  
4      10560
2      10505
1      10505
3      10505
0      10505
Name: count, Length: 481, dtype: int64

In [5]:
targets = train.target.value_counts(sort=True)
targets

target
-0.159740      3477
-0.759959      3471
-0.050068      3470
-0.079870      3467
 1.109838      3459
               ... 
 62.299965        1
 59.139730        1
-70.880060        1
-108.349920       1
-72.960260        1
Name: count, Length: 15934, dtype: int64

In [6]:
len(train)

5237980

In [7]:
import importlib

In [8]:
importlib.reload(torch_classes)
trading_data = torch_classes.TradingData(train)
trading_data.generate_batches()
model = torch_classes.GRUNet(12,32)
X = trading_data.packed_x[0]
Y = trading_data.packed_y[0]

 10%|▉         | 19/200 [00:09<01:25,  2.11it/s]

Missing Targets for day=438,for stock_id=19, Excluding


 50%|█████     | 101/200 [00:44<00:35,  2.77it/s]

Missing Targets for day=328,for stock_id=101, Excluding


 66%|██████▌   | 131/200 [00:58<00:47,  1.45it/s]

...

 79%|███████▉  | 158/200 [01:09<00:14,  2.82it/s]

Missing Targets for day=388,for stock_id=158, Excluding


100%|██████████| 200/200 [01:29<00:00,  2.23it/s]


Length of train: 385, Length of test 96


100%|██████████| 95/95 [00:00<00:00, 11864.90it/s]
c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [9]:
data = train

In [10]:
# data_grouped_stock_id = data.groupby('stock_id')
# for stock_id,stock_data in tqdm(data_grouped_stock_id):

#     stock_daily_group = stock_data.groupby('date_id')

#     for day, stock_daily_data in stock_daily_group:
#         if stock_daily_data['target'].isna().sum():
#             print(f'Missing Targets for {day=},for {stock_id=}, Excluding')
#             data.drop(stock_daily_data.index)
#             continue
    # data_grouped_daily = data.groupby('date_id')

In [11]:
trading_df = trading_data

In [12]:
model = torch_classes.GRUNet(12,32).to('cuda:0')

In [13]:
stocks = [trading_df.stocksDict[x] for x in trading_df.stock_batches[0]]
stock_data = trading_df.train_batches
X = trading_data.packed_x[0]
Y = trading_data.packed_y[0].data

hidden_in = torch.stack([x.hidden for x in stocks])

output,hidden = model(X)
hidden = hidden.transpose(0,1)
output  = torch.flatten(output)

[setattr(obj, 'hidden', val) for obj, val in zip(stocks,hidden)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [14]:
stocks = [trading_df.stocksDict[x] for x in trading_df.stock_batches[0]]
len(stocks)

191

In [15]:
torch.zeros(31,1).shape

torch.Size([31, 1])

In [16]:
hidden.shape

torch.Size([191, 1, 32])

In [17]:
trading_df.stocksDict[0].hidden.shape

torch.Size([1, 32])

In [18]:
trading_df.reset_hidden()

In [19]:
trading_df.stocksDict[0].hidden.shape

torch.Size([1, 32])

In [20]:
def train_model(trading_df:torch_classes.TradingData, model:torch_classes.GRUNet, config:dict):
    with wandb.init(project="Optviver", config=config,):
            example_ct = 0
            epochs = 10000
            num_batches = len(trading_df.train_batches)-2
            criterion = nn.L1Loss()
            model = model.to('cuda:0')
            optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
            trading_df.reset_hidden()
            for epoch in trange(epochs):
                model.train()
                loss_list = []
                
                for i in range(0,384):
                    # print(i)

                    stocks = [trading_df.stocksDict[x] for x in trading_df.stock_batches[i]] #Stocks for the Day
                    stock_data = trading_df.train_batches

                    example_ct+=1

                    X = trading_data.packed_x[i]
                    Y = trading_data.packed_y[i].data

                    hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

                    output,hidden = model(X,hidden_in)
                    hidden = hidden.transpose(0,1)
                    output  = torch.flatten(output)

                    [setattr(obj, 'hidden', val.detach()) for obj, val in zip(stocks,hidden)]

                    if output.shape!=Y.shape:
                         print(i,output.shape,Y.shape,)

                    loss = criterion(output,Y)
                    # print(loss)

                    loss.backward()
                    loss_list.append((i,loss))
                    if loss.isnan():
                         continue
                    # print(loss)
                    optimizer.step()
                    if i == 0:
                        epoch_loss = loss
                        # print(epoch_loss)
                    else:
                        if loss.isnan():
                            pass
                        epoch_loss = loss+epoch_loss

                    wandb.log({"loss_1": torch.mean(loss).item(), 'epoch':epoch}, step = example_ct)
                    trading_df.detach_hidden()
                validate_model(trading_df,model,criterion,epoch)
                trading_df.reset_hidden()
                # print(epoch_loss)
                # print(loss_list)
                wandb.log({"epoch_loss": epoch_loss/384, 'epoch':epoch}, step = example_ct)

@torch.no_grad()          
def validate_model(trading_df:torch_classes.TradingData,model:torch_classes.GRUNet,criterion,epoch,):
    model.eval()
    val_batches = trading_df.packed_val_x
    len_val = len(val_batches)

    for i in trange(0,len_val-1):
        stocks = [trading_df.stocksDict[x] for x in trading_df.stock_batches[0]] 
         
        X = trading_df.packed_val_x[i]
        Y = trading_df.packed_val_y[i].data

        hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)

        output,hidden = model(X,hidden_in)
        hidden = hidden.transpose(0,1)
        output  = torch.flatten(output)

        # [setattr(obj, 'hidden', val) for obj, val in zip(stocks,hidden)]

        loss = criterion(output,Y)

        # if loss.isnan():
        #     continue

        if i == 0:
            epoch_loss = loss
            # print(epoch_loss)
        else:
            # if loss.isnan():
                # pass
            epoch_loss = loss+epoch_loss

    wandb.log({"val_epoch_loss": epoch_loss/len_val, 'epoch':epoch})


    
     







In [21]:
# trading_df.detach_hidden()
# trading_df.reset_hidden()

In [22]:
config_static = {'learning_rate':0.0001}

In [23]:
CUDA_LAUNCH_BLOCKING=1
train_model(trading_data,model,config=config_static)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: nickojelly. Use `wandb login --relogin` to force relogin


  0%|          | 0/10000 [00:03<?, ?it/s]
Traceback (most recent call last):
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_17380\4287905476.py", line 55, in train_model
    validate_model(trading_df,model,criterion,epoch)
  File "c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\utils\_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Nick\AppData\Local\Temp\ipykernel_17380\4287905476.py", line 73, in validate_model
    hidden_in = torch.stack([x.hidden for x in stocks]).transpose(0,1)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
loss_1,▂▂▁▂▁▁▂▃▅▃▂▄█▅▄▄▃▄▃▆▆▃▃▃▂▂▂▁▂▆▂▃▃▄▂▄▂▃▃▃
epoch,0
loss_1,6.364


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
